# CNN
## Increase the complexity of the CNN and Ensemble methods
#### Ensemble methods: Instead of training a single model, you can try using ensemble methods such as bagging or boosting. These techniques involve training multiple models and combining their predictions, which often leads to better performance.

In [9]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf

# Load the training data and labels
df_features = pd.read_csv('traindata.txt', delimiter=',', header=None)
df_labels = pd.read_csv('trainlabels.txt', header=None)

# Split df_features into X_train and X_test
X_train, X_test, y_train, y_test = train_test_split(
    df_features,
    df_labels,
    test_size=0.3,
    random_state=42
)

print(df_features.shape)
print(df_labels.shape)

# Assuming X_train is a Pandas Series
# Data augmentation - random perturbations
augmented_X_train = []
augmented_y_train = []

for i in range(len(X_train)):
    original_data = X_train.iloc[i].to_numpy()
    augmented_X_train.append(original_data)
    augmented_y_train.append(y_train.iloc[i].values[0])

    # Apply random perturbations
    perturbed_data = original_data + np.random.normal(0, 0.1, size=original_data.shape)
    augmented_X_train.append(perturbed_data)
    augmented_y_train.append(y_train.iloc[i].values[0])

# Convert augmented data to DataFrames
augmented_X_train = pd.DataFrame(augmented_X_train)
augmented_y_train = pd.DataFrame(augmented_y_train)

# Concatenate augmented data with original data
X_train = pd.concat([X_train, augmented_X_train], axis=0)
y_train = pd.concat([y_train, augmented_y_train], axis=0)

# Shuffle the augmented data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Reshape the data for CNN
X_train = X_train.values.reshape(-1, 71, 1)
X_test = X_test.values.reshape(-1, 71, 1)

# Define some constants
INPUT_SHAPE = (71, 1)  # Number of input features and channels
NUM_CLASSES = 10  # Number of output classes (0-9)
LEARNING_RATE = 0.001  # Adjust as necessary
NUM_MODELS = 1  # Number of models in the ensemble

# One-hot encoding of output
y_train_encoded = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test_encoded = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

# Define the ensemble of models
ensemble_models = []
for _ in range(NUM_MODELS):
    # Define the CNN architecture
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=INPUT_SHAPE),
        tf.keras.layers.MaxPooling1D(pool_size=2),

        tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation='relu'),

        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])

    # Define an Adam optimizer with the desired learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

    # Compile the model with the custom optimizer
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    

    # Append the model to the ensemble
    ensemble_models.append(model)

# Train each model in the ensemble
for model_index, model in enumerate(ensemble_models):
    print(f"Model {model_index + 1} Training:")
    history = model.fit(X_train, y_train_encoded,
                        epochs=30,
                        batch_size=32,
                        verbose=1)  # Set verbose=1 to see training progress

    # Predicting the test set results
    y_test_pred_prob = model.predict(X_test)
    y_test_pred = np.argmax(y_test_pred_prob, axis=1)

    # Summary of the model
    model.summary()

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_test_pred)
    print('Model Accuracy:', accuracy)
    print('Model Loss:', history.history['loss'][-1])
    print()

# Predicting the test set results using the ensemble
y_test_preds = []
for model in ensemble_models:
    y_test_pred_prob = model.predict(X_test)
    y_test_pred = np.argmax(y_test_pred_prob, axis=1)
    y_test_preds.append(y_test_pred)

# Take the majority vote from the ensemble predictions
y_test_preds_ensemble = np.round(np.mean(y_test_preds, axis=0)).astype(int)

# Calculate accuracy of the ensemble model
ensemble_accuracy = accuracy_score(y_test, y_test_preds_ensemble)
print('Ensemble Model Accuracy:', ensemble_accuracy)


(10000, 71)
(10000, 1)
Model 1 Training:
Epoch 1/30
657/657 [==============================] - 2s 2ms/step - loss: 2.2535 - accuracy: 0.1460
Epoch 2/30
657/657 [==============================] - 2s 2ms/step - loss: 2.1251 - accuracy: 0.2115
Epoch 3/30
657/657 [==============================] - 2s 2ms/step - loss: 2.0086 - accuracy: 0.2608
Epoch 4/30
657/657 [==============================] - 2s 2ms/step - loss: 1.8951 - accuracy: 0.3075
Epoch 5/30
657/657 [==============================] - 2s 2ms/step - loss: 1.7780 - accuracy: 0.3473
Epoch 6/30
657/657 [==============================] - 2s 2ms/step - loss: 1.6656 - accuracy: 0.3890
Epoch 7/30
657/657 [==============================] - 2s 2ms/step - loss: 1.5670 - accuracy: 0.4295
Epoch 8/30
657/657 [==============================] - 2s 2ms/step - loss: 1.4826 - accuracy: 0.4550
Epoch 9/30
657/657 [==============================] - 2s 2ms/step - loss: 1.4041 - accuracy: 0.4852
Epoch 10/30
657/657 [==============================] - 2s 3

In [29]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Rotate Data Function
def rotate_data(data, angle):
    rotated_data = []
    for image in data:
        rotated_image = np.concatenate((image[angle:], image[:angle]))
        rotated_data.append(rotated_image)
    return np.array(rotated_data)

# Add Noise Function
def add_noise(data, mean, std_dev):
    noisy_data = data + np.random.normal(mean, std_dev, size=data.shape)
    return noisy_data

# Read the data
df_features = pd.read_csv('traindata.txt', delimiter=',', header=None)
df_labels = pd.read_csv('trainlabels.txt', header=None)

# Split df_features into X_train and X_test
X_train, X_test, y_train, y_test = train_test_split(
    df_features,
    df_labels,
    test_size=0.3,
    random_state=42
)

print(df_features.shape)
print(df_labels.shape)

# Normalize the input data using Min-Max scaling
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Data augmentation - random perturbations
augmented_X_train = []
augmented_y_train = []

for i in range(len(X_train)):
    original_data = X_train[i]
    augmented_X_train.append(original_data)
    augmented_y_train.append(y_train.iloc[i].values[0])

    # Apply random perturbations
    perturbed_data = original_data + np.random.normal(0, 0.1, size=original_data.shape)
    augmented_X_train.append(perturbed_data)
    augmented_y_train.append(y_train.iloc[i].values[0])

# Convert augmented data to NumPy arrays
X_train = np.array(augmented_X_train)
y_train = np.array(augmented_y_train)

# Shuffle the augmented data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Rotate the data
X_train = rotate_data(X_train, angle=10)
X_test = rotate_data(X_test, angle=10)

# Add noise to the data
X_train = add_noise(X_train, 0, 0.1)
X_test = add_noise(X_test, 0, 0.1)

# Define some constants
INPUT_SHAPE = (71,)  # Number of input features
NUM_CLASSES = 10  # Number of output classes (0-9)
LEARNING_RATE = 0.001  # Adjust as necessary

# One-hot encoding of output
y_train_encoded = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test_encoded = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

# Define the NN architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=INPUT_SHAPE),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),  # Additional hidden layer
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),  # Additional hidden layer
    tf.keras.layers.Dense(64, activation='relu'),  # Additional hidden layer
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Define an Adam optimizer with the desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# Compile the model with the custom optimizer
model.compile(optimizer=optimizer,
              loss='mean_squared_error',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(X_train, y_train_encoded,
                    epochs=30,
                    batch_size=64)

# Predicting the test set results
y_test_pred_prob = model.predict(X_test)
y_test_pred = np.argmax(y_test_pred_prob, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print('Model accuracy:', accuracy)


(10000, 71)
(10000, 1)
Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_123 (Dense)           (None, 64)                4608      
                                                                 
 dense_124 (Dense)           (None, 128)               8320      
                                                                 
 dense_125 (Dense)           (None, 256)               33024     
                                                                 
 dropout_19 (Dropout)        (None, 256)               0         
                                                                 
 dense_126 (Dense)           (None, 128)               32896     
                                                                 
 dense_127 (Dense)           (None, 64)                8256      
                                                                 
 dense_128 (Dense)           (

In [68]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


# Rotate Data Function
def rotate_data(data, angle):
    rotated_data = []
    for _, row in data.iterrows():
        image = row.to_numpy().reshape((71,))
        # Perform rotation operation specific to your data shape
        # Modify the rotation operation based on your specific requirements
        rotated_image = np.concatenate((image[angle:], image[:angle]))
        rotated_data.append(rotated_image)
    return np.array(rotated_data)

# Add Noise Function
def add_noise(data, mean, std_dev):
    noisy_data = data + np.random.normal(mean, std_dev, size=data.shape)
    return noisy_data



# Read the data
df_features = pd.read_csv('traindata.txt', delimiter=',', header=None)
df_labels = pd.read_csv('trainlabels.txt', header=None)

# Split df_features into X_train and X_test
X_train, X_test, y_train, y_test = train_test_split(
    df_features,
    df_labels,
    test_size=0.3,
    random_state=42
)

print(df_features.shape)
print(df_labels.shape)

# Data augmentation - random perturbations
augmented_X_train = []
augmented_y_train = []

for i in range(len(X_train)):
    original_data = X_train.iloc[i].to_numpy()
    augmented_X_train.append(original_data)
    augmented_y_train.append(y_train.iloc[i].values[0])

    # Apply random perturbations
    perturbed_data = original_data + np.random.normal(0, 0.1, size=original_data.shape)
    augmented_X_train.append(perturbed_data)
    augmented_y_train.append(y_train.iloc[i].values[0])

# Convert augmented data to DataFrames
augmented_X_train = pd.DataFrame(augmented_X_train)
augmented_y_train = pd.DataFrame(augmented_y_train)

# Concatenate augmented data with original data
X_train = pd.concat([X_train, augmented_X_train], axis=0)
y_train = pd.concat([y_train, augmented_y_train], axis=0)

# Shuffle the augmented data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Rotate the data
X_train = rotate_data(X_train, angle=10)
X_test = rotate_data(X_test, angle=10)


# Add noise to the data
X_train = add_noise(X_train, 0, 0.1)
X_test = add_noise(X_test, 0, 0.1)


# Define some constants
INPUT_SHAPE = (71,)  # Number of input features
NUM_CLASSES = 10  # Number of output classes (0-9)
LEARNING_RATE = 0.025  # Adjust as necessary

# One-hot encoding of output
y_train_encoded = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test_encoded = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

# Define the NN architecture
# Define the NN architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=INPUT_SHAPE),
    tf.keras.layers.BatchNormalization(),  # Add batch normalization
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),  # Add batch normalization
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256, activation='relu'),  # Additional hidden layer
    tf.keras.layers.BatchNormalization(),  # Add batch normalization
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),  # Additional hidden layer
    tf.keras.layers.BatchNormalization(),  # Add batch normalization
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),  # Additional hidden layer
    tf.keras.layers.BatchNormalization(),  # Add batch normalization
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])


# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(64, input_shape=INPUT_SHAPE),
#     tf.keras.layers.LeakyReLU(alpha=0.2),  # Replace activation with LeakyReLU
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dense(128),
#     tf.keras.layers.ELU(alpha=0.2),  # Replace activation with LeakyReLU
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(256),
#     tf.keras.layers.LeakyReLU(alpha=0.2),  # Replace activation with LeakyReLU
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(128),
#     tf.keras.layers.LeakyReLU(alpha=0.2),  # Replace activation with LeakyReLU
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(64),
#     tf.keras.layers.LeakyReLU(alpha=0.2),  # Replace activation with LeakyReLU
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
# ])


# Define an Adam optimizer with the desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# Compile the model with the custom optimizer
model.compile(optimizer=optimizer,
              loss='mean_squared_error',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(X_train, y_train_encoded,
                    epochs=30,
                    batch_size=64)

# Predicting the test set results
y_test_pred_prob = model.predict(X_test)
y_test_pred = np.argmax(y_test_pred_prob, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print('Model accuracy:', accuracy)

(10000, 71)
(10000, 1)
Model: "sequential_62"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_359 (Dense)           (None, 64)                4608      
                                                                 
 batch_normalization_137 (Ba  (None, 64)               256       
 tchNormalization)                                               
                                                                 
 dense_360 (Dense)           (None, 128)               8320      
                                                                 
 batch_normalization_138 (Ba  (None, 128)              512       
 tchNormalization)                                               
                                                                 
 flatten_161 (Flatten)       (None, 128)               0         
                                                                 
 dropout_135 (Dropout)       (